Collaborative Filtering
* Item-Item 
* Centered Cosine Similarity
* Top-k

https://www.ethanrosenthal.com/2015/11/02/intro-to-collaborative-filtering/

## Import Data 

In [1]:
%matplotlib inline

import pandas as pd

r = pd.read_csv( 'ratings.csv' )

In [2]:
n_users = r.user_id.unique().shape[0]
n_books = r.book_id.unique().shape[0]
print(str(n_users) + ' users')
print(str(n_books) + ' books')

53424 users
10000 books


## Construct Matrix

In [3]:
import numpy as np

# id starts from 1, but python starts at 0
ratings = np.zeros((n_books, n_users))
for row in r.itertuples():
    ratings[row[2]-1, row[1]-1] = row[3]
    
ratings

array([[0., 0., 0., ..., 4., 4., 4.],
       [0., 5., 0., ..., 5., 5., 5.],
       [0., 0., 0., ..., 0., 0., 4.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [4]:
def get_sparsity(ratings):
    sparsity = float(len(ratings.nonzero()[0]))
    sparsity /= (ratings.shape[0] * ratings.shape[1])
    sparsity *= 100
    return sparsity

In [5]:
print(f'Sparsity: {get_sparsity(ratings)}%')

Sparsity: 1.118688042827194%


## Only keep top 1000 books

In [6]:
N = 1000
book_index = np.argpartition(np.count_nonzero(ratings, axis=1), -N)[-N:]
print(len(book_index))

1000


In [7]:
ratings = ratings[book_index]

In [8]:
book_stat = np.count_nonzero(ratings, axis=1)
print(f'min:  {book_stat.min()}\nmean: {book_stat.mean()}\nmax:  {book_stat.max()}')

min:  1183
mean: 3179.65
max:  22806


In [9]:
print(f'Sparsity: {get_sparsity(ratings)}%')

Sparsity: 5.951725816112609%


## Only keep top 10000 users

In [10]:
N = 10000
user_index = np.argpartition(np.count_nonzero(ratings, axis=0), -N)[-N:]
print(len(user_index))

10000


In [11]:
ratings = ratings[:, user_index]
ratings.shape

(1000, 10000)

In [12]:
user_stat = np.count_nonzero(ratings, axis=0)
print(f'min:  {user_stat.min()}\nmean: {user_stat.mean()}\nmax:  {user_stat.max()}')

min:  82
mean: 98.3166
max:  177


In [13]:
print(f'Sparsity: {get_sparsity(ratings)}%')

Sparsity: 9.831660000000001%


## Get Train and Test Set

Removing 10 ratings per item from the training set and placing them in the test set.

In [14]:
def train_test_split(ratings):
    test = np.zeros(ratings.shape)
    train = ratings.copy()
    for book in range(ratings.shape[0]):
        test_ratings = np.random.choice(ratings[book, :].nonzero()[0], 
                                        size=20, 
                                        replace=False)
        train[book, test_ratings] = 0
        test[book, test_ratings] = ratings[book, test_ratings]
        
    # Test and training are truly disjoint
    assert(np.all((train * test) == 0)) 
    return train, test

train, test = train_test_split(ratings)

## Center Matrix

In [15]:
train_centered = train.copy()

def center_row(row):
    index = row.nonzero()[0]
    avg = row[index].mean()
    row[index] = row[index] - avg
    return row
    
train_centered = np.apply_along_axis(center_row, 1, train_centered)
train_centered

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-1.38375209,  0.61624791, -0.38375209, ...,  0.        ,
        -0.38375209,  0.61624791],
       [-0.0108742 , -0.0108742 , -1.0108742 , ...,  0.9891258 ,
         0.        ,  0.9891258 ],
       [ 0.71932831,  0.71932831, -1.28067169, ..., -0.28067169,
         0.        , -0.28067169]])

## Calculate Similarity Matrix

In [16]:
def similarity(ratings, kind='item'):
    if kind == 'item':
        sim = ratings.dot(ratings.T)
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return (sim / norms / norms.T)

item_similarity = similarity(train_centered, kind='item')

## Calculate MSE Error

In [17]:
from sklearn.metrics import mean_squared_error

def get_mse(pred, actual):
    # Ignore nonzero terms
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

## Predict Result (Slow because of for loop)

In [18]:
def predict(item_similarity, train, K=40):
    pred = np.zeros(train.shape)
    for i in range(train.shape[0]):
        for j in range(train.shape[1]):
            # Already rated by user
            if (train[i][j] != 0):
                pred[i][j] = train[i][j]
                continue
            # Note rated by user
            user_nonzero = np.where(train[:, j] != 0)[0]
            similarity = item_similarity[i][user_nonzero]
            topK = np.argpartition(similarity, -K)[-K:]
            pred[i][j] = np.dot(similarity[topK], train[user_nonzero[topK], j]) / similarity[topK].sum()
    return pred

In [19]:
result = predict(item_similarity, train)

## Calculate MSE

In [20]:
print('Top-k Item-based CF MSE: ' + str(get_mse(result, test)))

Top-k Item-based CF MSE: 0.7394827248443676
